In [5]:
import requests
from bs4 import BeautifulSoup
import time
import random

session = requests.Session()

# Step 1: GET /price page first
url = 'https://kalimatimarket.gov.np/price'
response = session.get(url)

# Step 2: Extract _token
soup = BeautifulSoup(response.text, 'html.parser')
token = soup.find('input', {'name': '_token'})['value']
print(f'CSRF token: {token}')

# Step 3: Now POST with token + datePricing
post_data = {
    '_token': token,
    'datePricing': '2025-06-07'  # example date
}

response_post = session.post(url, data=post_data)
soup_post = BeautifulSoup(response_post.text, 'html.parser')

# Example table parsing
table = soup_post.find('table')
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    data_row = [col.text.strip() for col in cols]
    print(data_row)


CSRF token: loaXSuNS9TjVNhbHKiXdxETygvUbO9gKlZ4ztzAz
['गोलभेडा ठूलो(नेपाली)', 'के.जी.', 'रू ६५.००', 'रू ७५.००', 'रू ७०.००']
['गोलभेडा ठूलो(भारतीय)', 'केजी', 'रू ७०.००', 'रू ८०.००', 'रू ७५.००']
['गोलभेडा सानो(लोकल)', 'के.जी.', 'रू ३९.००', 'रू ४८.००', 'रू ४३.६७']
['गोलभेडा सानो(टनेल)', 'के जी', 'रू ४८.००', 'रू ६०.००', 'रू ५४.००']
['गोलभेडा सानो(भारतीय)', 'के जी', 'रू ४०.००', 'रू ५०.००', 'रू ४५.००']
['आलु रातो', 'के.जी.', 'रू ३७.००', 'रू ४०.००', 'रू ३८.३३']
['आलु रातो(भारतीय)', 'के जी', 'रू ३२.००', 'रू ३५.००', 'रू ३३.३३']
['आलु रातो(मुडे)', 'केजी', 'रू ३८.००', 'रू ४०.००', 'रू ३९.००']
['प्याज सुकेको (भारतीय)', 'के.जी.', 'रू ३८.००', 'रू ४०.००', 'रू ३९.००']
['गाजर(लोकल)', 'के.जी.', 'रू ४५.००', 'रू ५५.००', 'रू ५०.००']
['बन्दा(लोकल)', 'के.जी.', 'रू १५.००', 'रू २०.००', 'रू १७.६७']
['काउली स्थानिय', 'के.जी.', 'रू ४०.००', 'रू ५०.००', 'रू ४५.००']
['मूला रातो', 'के.जी.', 'रू ३०.००', 'रू ३५.००', 'रू ३२.३३']
['मूला सेतो(लोकल)', 'के.जी.', 'रू १५.००', 'रू २०.००', 'रू १७.६७']
['सेतो मूला(हाइब्रीड)', 'के

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
from datetime import datetime, timedelta

# Function to generate dates between start and end date (inclusive)
def generate_dates(start_date, end_date):
    current = start_date
    while current <= end_date:
        yield current.strftime('%Y-%m-%d')
        current += timedelta(days=1)

session = requests.Session()
url = 'https://kalimatimarket.gov.np/price'

# Define your date range here
start_date = datetime.strptime('2024-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2025-01-01', '%Y-%m-%d')

# Open CSV file once, write headers and append data
with open('kalimati_prices_nepali.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = None

    for date_str in generate_dates(start_date, end_date):
        # Step 1: GET page to get fresh CSRF token
        response = session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        token = soup.find('input', {'name': '_token'})['value']

        # Step 2: POST request with date and token
        post_data = {
            '_token': token,
            'datePricing': date_str
        }
        response_post = session.post(url, data=post_data)
        soup_post = BeautifulSoup(response_post.text, 'html.parser')

        # Step 3: Parse table
        table = soup_post.find('table')
        print(table)
        if table is None:
            print(f'No table found for date {date_str}, skipping...')
            continue

        rows = table.find_all('tr')
        if not rows or len(rows) < 2:
            print(f'No data rows for date {date_str}, skipping...')
            continue

        # Write headers if first iteration
        if writer is None:
            headers = [th.text.strip() for th in rows[0].find_all('th')]
            writer = csv.writer(csvfile)
            # Add 'Date' column for reference
            writer.writerow(['Date'] + headers)

        # Write data rows
        for row in rows[1:]:
            cols = [col.text.strip() for col in row.find_all('td')]
            writer.writerow([date_str] + cols)

        print(f'Data for {date_str} saved.')
    
        # Delay between requests (random between 1 to 3 seconds)
        time.sleep(random.uniform(1, 3))

print('All done.')


<table class="display dt-large" id="commodityPriceParticular" style="width:100%;"><thead><tr><th>कृषि उपज</th><th>ईकाइ</th><th>न्यूनतम</th><th>अधिकतम</th><th>औसत</th></tr></thead><tbody><tr><td>गोलभेडा ठूलो(नेपाली)</td><td>के.जी.</td><td>रू ५०.००</td><td>रू ६०.००</td><td>रू ५५.००</td></tr><tr><td>गोलभेडा ठूलो(भारतीय)</td><td>केजी</td><td>रू ६०.००</td><td>रू ७०.००</td><td>रू ६५.००</td></tr><tr><td>गोलभेडा सानो(लोकल)</td><td>के.जी.</td><td>रू १५.००</td><td>रू २५.००</td><td>रू १९.००</td></tr><tr><td>गोलभेडा सानो(टनेल)</td><td>के जी</td><td>रू २५.००</td><td>रू ३५.००</td><td>रू ३०.००</td></tr><tr><td>गोलभेडा सानो(भारतीय)</td><td>के जी</td><td>रू ३०.००</td><td>रू ३५.००</td><td>रू ३२.५०</td></tr><tr><td>गोलभेडा सानो(तराई)</td><td>के जी</td><td>रू २०.००</td><td>रू ३०.००</td><td>रू २५.००</td></tr><tr><td>आलु रातो</td><td>के.जी.</td><td>रू ३५.००</td><td>रू ३८.००</td><td>रू ३६.४०</td></tr><tr><td>आलु रातो(भारतीय)</td><td>के जी</td><td>रू ३०.००</td><td>रू ३२.००</td><td>रू ३१.००</td></tr><tr><td>आल

ReadTimeout: HTTPSConnectionPool(host='kalimatimarket.gov.np', port=443): Read timed out. (read timeout=None)

In [22]:
nepali_to_english_dict = {
    "गोलभेडा ठूलो(नेपाली)": "Tomato Big(Nepali)",
    "गोलभेडा ठूलो(भारतीय)": "Tomato Big(Indian)",
    "गोलभेडा सानो(लोकल)": "Tomato Small(Local)",
    "गोलभेडा सानो(टनेल)": "Tomato Small(Tunnel)",
    "गोलभेडा सानो(भारतीय)": "Tomato Small(Indian)",
    "गोलभेडा सानो(तराई)": "Tomato Small(Terai)",
    "आलु रातो": "Potato Red",
    "आलु रातो(भारतीय)": "Potato Red(Indian)",
    "आलु रातो(मुडे)": "Potato Red(Mude)",
    "आलु सेतो": "Potato White",
    "प्याज सुकेको (भारतीय)": "Onion Dry (Indian)",
    "प्याज सुकेको (चाइनिज)": "Onion Dry (Chinese)",
    "गाजर(लोकल)": "Carrot(Local)",
    "गाजर(तराई)": "Carrot(Terai)",
    "बन्दा(लोकल)": "Cabbage(Local)",
    "बन्दा": "Cabbage",
    "बन्दा(तराई)": "Cabbage(Terai)",
    "काउली स्थानिय": "Cauli Local",
    "काउली स्थानिय(ज्यापु)": "Cauli Local(Jyapu)",
    "रायो साग": "Spinach Leaf",
    "पालूगो साग": "Cress Leaf",
    "चमसूरको साग": "Mustard Leaf",
    "तोरीको साग": "Mustard Leaf",
    "मेथीको साग": "Fenugreek Leaf",
    "काउली(तराई)": "Cauli Terai",
    "मूला रातो": "Raddish Red",
    "मूला सेतो(लोकल)": "Raddish White(Local)",
    "सेतो मूला(हाइब्रीड)": "Raddish White(Hybrid)",
    "भन्टा लाम्चो": "Brinjal Long",
    "भन्टा डल्लो": "Brinjal Round",
    "बोडी(तने)": "Cow pea(Long)",
    "बोडी(छोटो)": "Cowpea(Short)",
    "मटरकोशा": "Green Peas",
    "घिउ सिमी(लोकल)": "French Bean(Local)",
    "घिउ सिमी(हाइब्रीड)": "French Bean(Hybrid)",
    "घिउ सिमी(राजमा)": "French Bean(Rajma)",
    "भटमासकोशा": "Soyabean Green",
    "तितो करेला": "Bitter Gourd",
    "लौका": "Bottle Gourd",
    "परवर(लोकल)": "Pointed Gourd(Local)",
    "परवर(तराई)": "Pointed Gourd(Terai)",
    "चिचिण्डो": "Snake Gourd",
    "घिरौला": "Smooth Gourd",
    "झिगूनी": "Sponge Gourd",
    "फर्सी पाकेको": "Pumpkin",
    "फर्सी हरियो(लाम्चो)": "Squash(Long)",
    "हरियो फर्सी(डल्लो)": "Squash(Round)",
    "गान्टे मूला": "Turnip A",
    "तोरीको मुन्टा": "Turnip",
    "भिण्डी": "Okara",
    "सखरखण्ड": "Christophine",
    "पिंडालू": "Brd Leaf Mustard",
    "प्याज हरियो": "Onion Green",
    "च्याउ(कन्य)": "Mushroom(Kanya)",
    "च्याउ(डल्ले)": "Mushroom(Button)",
    "राजा च्याउ": "Arum",
    "सिताके च्याउ": "Mushroom(Shiitake)",
    "कुरीलो": "Asparagus",
    "न्यूरो": "Neuro",
    "ब्रोकाउली": "Brocauli",
    "चुकुन्दर": "Sugarbeet",
    "सजिवन": "Drumstick",
    "रातो बन्दा": "Red Cabbbage",
    "जिरीको साग": "Lettuce",
    "सेलरी": "Celery",
    "पार्सले": "Parseley",
    "सौफको साग": "Fennel Leaf",
    "पुदीना": "Mint",
    "इमली": "Tamarind",
    "तामा": "Bamboo Shoot",
    "तोफु": "Tofu",
    "गुन्दुक": "Gundruk",
    "स्याउ(झोले)": "Apple(Jholey)",
    "स्याउ(फूजी)": "Apple(Fuji)",
    "केरा": "Banana",
    "कागती": "Lime",
    "निबुवा": "Lemon",
    "अनार": "Pomegranate",
    "आँप(मालदह)": "Mango(Maldah)",
    "आँप(दसहरी)": "Mango(Dushari)",
    "आँप(कलकत्ते)": "Mango(Calcutte)",
    "आँप(चौसा)": "Mango(Chousa)",
    "अंगुर(हरियो)": "Grapes(Green)",
    "अंगुर(कालो)": "Grapes(Black)",
    "तरबुजा(हरियो)": "Water Melon(Green)",
    "तरबुजा(चित्ता परेको)": "Water Melon(Dotted)",
    "मौसम": "Sweet Orange",
    "सुन्तला(नेपाली)": "Orange(Nepali)",
    "सुन्तला(भारतीय)": "Orange(Indian)",
    "किन्नु": "Kinnow",
    "जुनार": "Pineapple",
    "भुई कटहर": "Jack Fruit",
    "रुख कटहर": "Papaya(Nepali)",
    "पपैया(भारतीय)": "Papaya(Indian)",
    "काक्रो(लोकल)": "Cucumber(Local)",
    "काक्रो(हाइब्रीड)": "Cucumber(Hybrid)",
    "नासपाती(चाइनिज)": "Pear(Chinese)",
    "मेवा(नेपाली)": "Pear(Local)",
    "मेवा(भारतीय)": "Pear(Indian)",
    "लीच्ची(लोकल)": "Litchi(Local)",
    "लीच्ची(भारतीय)": "Litchi(Indian)",
    "अदुवा": "Ginger",
    "खु्र्सानी सुकेको": "Chilli Dry",
    "खु्र्सानी हरियो": "Chilli Green",
    "खुर्सानी हरियो(बुलेट)": "Chilli Green(Bullet)",
    "खुर्सानी हरियो(माछे)": "Chilli Green(Machhe)",
    "खुर्सानी हरियो(अकबरे)": "Chilli Green(Akbare)",
    "भेडे खु्र्सानी": "Capsicum",
    "लसुन हरियो": "Garlic Green",
    "हरियो धनिया": "Coriander Green",
    "लसुन सुकेको चाइनिज": "Garlic Dry Chinese",
    "लसुन सुकेको नेपाली": "Garlic Dry Nepali",
    "छ्यापी सुकेको": "Clive Dry",
    "छ्यापी हरियो": "Clive Green",
    "मकै बोडी": "Maize",
    "सुगन्धित सुन्तला": "Sweet Lime",
    "अम्बा": "Guava",
    "जामुन": "Mombin",
    "बरेला": "Barela",
    "भण्टा तरकारी": "Sword Bean",
    "बाकुला": "Bakula",
    "गिठा": "Yam",
    "शकरकंद": "Sweet Potato",
    "म्यानडरिन": "Mandarin",
    "गाँठे कोबी": "Knolkhol",
    "स्ट्रबेरी": "Strawberry",
    "कोइरालो फूल": "Bauhania flower",
    "किवी": "Kiwi",
    "सरीफ": "Sarifa",
    "एभोकाडो": "Avocado",
    "अमला": "Amla",
    "ट्री टमाटर": "Tree Tomato",
    "किबी": "Kiwi",   # extra safety mapping for alt spelling
    "फिस फ्रेश": "Fish Fresh",
    "ताजा माछा(रहु)": "Fish Fresh(Rahu)",
    "ताजा माछा(बचुवा)": "Fish Fresh(Bachuwa)",
    "ताजा माछा(छडी)": "Fish Fresh(Chhadi)",
    "ताजा माछा(मुंगरी)": "Fish Fresh(Mungari)",
    "गन्ना": 'Sugarcane',
    'खरबुजा': 'Musk Melon'
}


In [31]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
from datetime import datetime, timedelta

def generate_dates(start_date, end_date):
    current = start_date
    while current <= end_date:
        yield current.strftime('%Y-%m-%d')
        current += timedelta(days=1)

session = requests.Session()
url = 'https://kalimatimarket.gov.np/price'

start_date = datetime.strptime('2023-09-28', '%Y-%m-%d')
end_date = datetime.strptime('2025-06-07', '%Y-%m-%d')

with open('kalimati_prices.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = None

    for date_str in generate_dates(start_date, end_date):
        response = session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        token = soup.find('input', {'name': '_token'})['value']

        post_data = {
            '_token': token,
            'datePricing': date_str
        }
        response_post = session.post(url, data=post_data)
        soup_post = BeautifulSoup(response_post.text, 'html.parser')

        table = soup_post.find('table')
        if table is None:
            print(f'No table found for date {date_str}, skipping...')
            continue

        rows = table.find_all('tr')
        if not rows or len(rows) < 2:
            print(f'No data rows for date {date_str}, skipping...')
            continue

        if writer is None:
            headers = [th.text.strip() for th in rows[0].find_all('th')]
            writer = csv.writer(csvfile)
            # Translate headers if needed - usually headers might be Nepali as well
            # So translate headers if they exist in dictionary
            translated_headers = []
            for h in headers:
                translated_headers.append(nepali_to_english_dict.get(h, h))
            writer.writerow(['Date'] + translated_headers)

        for row in rows[1:]:
            cols = [col.text.strip() for col in row.find_all('td')]

            # Translate the first column (Nepali item name) if possible
            # Assuming first column contains the Nepali product name to translate
            if len(cols) > 0:
                nepali_name = cols[0]
                english_name = nepali_to_english_dict.get(nepali_name, nepali_name)
                cols[0] = english_name

            for i in range(1, len(cols)):
                cols[i] = nepali_price_to_english(cols[i])
            
            writer.writerow([date_str] + cols)

        print(f'Data for {date_str} saved.')
        time.sleep(random.uniform(1, 3))

print('All done.')


Data for 2023-09-28 saved.
Data for 2023-09-29 saved.
Data for 2023-09-30 saved.
Data for 2023-10-01 saved.
Data for 2023-10-02 saved.
Data for 2023-10-03 saved.
Data for 2023-10-04 saved.
Data for 2023-10-05 saved.
Data for 2023-10-06 saved.
Data for 2023-10-07 saved.
Data for 2023-10-08 saved.
Data for 2023-10-09 saved.
Data for 2023-10-10 saved.
Data for 2023-10-11 saved.
Data for 2023-10-12 saved.
Data for 2023-10-13 saved.
Data for 2023-10-14 saved.
Data for 2023-10-15 saved.
Data for 2023-10-16 saved.
Data for 2023-10-17 saved.
Data for 2023-10-18 saved.
Data for 2023-10-19 saved.
Data for 2023-10-20 saved.
Data for 2023-10-21 saved.
Data for 2023-10-22 saved.
Data for 2023-10-23 saved.
Data for 2023-10-24 saved.
Data for 2023-10-25 saved.
Data for 2023-10-26 saved.
Data for 2023-10-27 saved.
Data for 2023-10-28 saved.
Data for 2023-10-29 saved.
Data for 2023-10-30 saved.
Data for 2023-10-31 saved.
Data for 2023-11-01 saved.
Data for 2023-11-02 saved.
Data for 2023-11-03 saved.
D

In [24]:
def nepali_price_to_english(price_str):
    # Remove currency symbol and spaces
    cleaned = price_str.replace('रू', '').strip()
    
    # Nepali digits mapping
    nepali_digits = '०१२३४५६७८९'
    english_digits = '0123456789'
    
    # Translate each Nepali digit to English digit
    translation_table = str.maketrans(''.join(nepali_digits), ''.join(english_digits))
    translated = cleaned.translate(translation_table)
    
    # Sometimes comma used as thousand separator - remove commas
    translated = translated.replace(',', '')
    
    # Convert to float string
    try:
        num = float(translated)
        return f"{num:.2f}"
    except ValueError:
        # If conversion fails, return original string
        return price_str


In [28]:

df_one = pd.read_csv('kalimati_tarkari_dataset_cleaned.csv')
commodity_unit_dict = dict(df_one[['Commodity', 'Unit']].drop_duplicates().values.tolist())

In [1]:
import pandas as pd
df_kali = pd.read_csv('kalimati_prices.csv')

In [36]:
unit_translation = {
    'केजी': 'kg',
    'के.जी.': 'kg',
    'के जी': 'kg',
    'के‍.जी': 'kg',
    'के.जी': 'kg',
    'दर्जन': 'dozen',
    'प्रति गोटा': 'per piece',
    'प्रति वटा': 'per piece'
}

In [37]:
df_kali['ईकाइ'] = df_kali['ईकाइ'].map(unit_translation)

In [38]:
df_kali

,Date,कृषि उपज,ईकाइ,न्यूनतम,अधिकतम,औसत
0,2023-09-28,Tomato Big(Indian),kg,40.0,50.0,45.0
1,2023-09-28,Tomato Small(Local),kg,50.0,60.0,55.0
2,2023-09-28,Tomato Small(Tunnel),kg,60.0,75.0,69.0
3,2023-09-28,Tomato Small(Indian),kg,45.0,55.0,50.0
4,2023-09-28,Potato Red,kg,56.0,58.0,57.0
...,...,...,...,...,...,...
62112,2025-06-07,Fish Fresh(Rahu),kg,310.0,330.0,320.0
62113,2025-06-07,Fish Fresh(Bachuwa),kg,300.0,320.0,310.0
62114,2025-06-07,Fish Fresh(Chhadi),kg,220.0,240.0,230.0
62115,2025-06-07,Arum,kg,280.0,300.0,290.0


In [39]:
df_kali= df_kali.rename(columns={
    'कृषि उपज': 'Commodity',
    'ईकाइ': 'Unit',
    'न्यूनतम': 'Minimum',
    'अधिकतम': 'Maximum',
    'औसत': 'Average'
})

In [40]:
df_kali

,Date,Commodity,Unit,Minimum,Maximum,Average
0,2023-09-28,Tomato Big(Indian),kg,40.0,50.0,45.0
1,2023-09-28,Tomato Small(Local),kg,50.0,60.0,55.0
2,2023-09-28,Tomato Small(Tunnel),kg,60.0,75.0,69.0
3,2023-09-28,Tomato Small(Indian),kg,45.0,55.0,50.0
4,2023-09-28,Potato Red,kg,56.0,58.0,57.0
...,...,...,...,...,...,...
62112,2025-06-07,Fish Fresh(Rahu),kg,310.0,330.0,320.0
62113,2025-06-07,Fish Fresh(Bachuwa),kg,300.0,320.0,310.0
62114,2025-06-07,Fish Fresh(Chhadi),kg,220.0,240.0,230.0
62115,2025-06-07,Arum,kg,280.0,300.0,290.0


In [4]:
df_kali['Commodity'].unique()

array(['Tomato Big(Indian)', 'Tomato Small(Local)',
       'Tomato Small(Tunnel)', 'Tomato Small(Indian)', 'Potato Red',
       'Potato Red(Indian)', 'Potato Red(Mude)', 'आलु सेतो',
       'Onion Dry (Indian)', 'Carrot(Local)', 'Carrot(Terai)',
       'Cabbage(Local)', 'Cauli Local', 'Raddish Red',
       'Raddish White(Local)', 'Raddish White(Hybrid)', 'Brinjal Long',
       'Brinjal Round', 'Cow pea(Long)', 'Maize', 'French Bean(Local)',
       'French Bean(Hybrid)', 'टाटे सिमी', 'Soyabean Green',
       'Bitter Gourd', 'Bottle Gourd', 'Pointed Gourd(Local)',
       'Pointed Gourd(Terai)', 'Snake Gourd', 'Smooth Gourd', 'Pumpkin',
       'Squash(Round)', 'Okara', 'बरेला', 'Brd Leaf Mustard', 'स्कूस',
       'Spinach Leaf', 'Mustard Leaf', 'Onion Green', 'Mushroom(Kanya)',
       'Mushroom(Button)', 'Asparagus', 'Neuro', 'Brocauli', 'Sugarbeet',
       'Red Cabbbage', 'Lettuce', 'Celery', 'Parseley', 'Mint',
       'Turnip A', 'Tamarind', 'Bamboo Shoot', 'Tofu', 'Gundruk',
       'App

In [6]:
len(nepali_to_english_dict)

102

In [12]:
df_kali[df_kali['Commodity'].str.]

,Unnamed: 0,Date,Commodity,Unit,Minimum,Maximum,Average
19,19,2023-09-28,Maize,kg,80.0,90.0,86.67
30,30,2023-09-28,Pumpkin,kg,45.0,50.0,48.33
32,32,2023-09-28,Okara,kg,45.0,55.0,50.00
41,41,2023-09-28,Asparagus,kg,450.0,500.0,475.00
42,42,2023-09-28,Neuro,kg,80.0,90.0,85.00
...,...,...,...,...,...,...,...
62083,62083,2025-06-07,Pomegranate,kg,300.0,350.0,326.67
62090,62090,2025-06-07,Pineapple,kg,200.0,250.0,226.67
62099,62099,2025-06-07,Ginger,kg,90.0,110.0,100.00
62105,62105,2025-06-07,Capsicum,kg,80.0,90.0,85.00


In [13]:
english_list = [
    'Tomato Big(Nepali)', 'Tomato Big(Indian)', 'Tomato Small(Local)', 'Tomato Small(Tunnel)', 'Tomato Small(Indian)',
    'Potato Red', 'Potato Red(Indian)', 'Potato Red(Mude)', 'Onion Dry (Indian)', 'Carrot(Local)', 'Carrot(Terai)',
    'Cabbage(Local)', 'Cauli Local', 'Raddish Red', 'Raddish White(Local)', 'Raddish White(Hybrid)', 'Brinjal Long',
    'Brinjal Round', 'Cow pea(Long)', 'Maize', 'Green Peas', 'French Bean(Local)', 'French Bean(Hybrid)',
    'French Bean(Rajma)', 'Soyabean Green', 'Bitter Gourd', 'Bottle Gourd', 'Pointed Gourd(Local)', 'Pointed Gourd(Terai)',
    'Snake Gourd', 'Smooth Gourd', 'Sponge Gourd', 'Pumpkin', 'Squash(Long)', 'Squash(Round)', 'Okara', 'Christophine',
    'Brd Leaf Mustard', 'Spinach Leaf', 'Cress Leaf', 'Mustard Leaf', 'Fenugreek Leaf', 'Onion Green', 'Mushroom(Kanya)',
    'Mushroom(Button)', 'Asparagus', 'Neuro', 'Brocauli', 'Sugarbeet', 'Drumstick', 'Red Cabbbage', 'Lettuce', 'Cabbage',
    'Celery', 'Parseley', 'Fennel Leaf', 'Mint', 'Turnip A', 'Tamarind', 'Bamboo Shoot', 'Tofu', 'Gundruk', 'Apple(Jholey)',
    'Apple(Fuji)', 'Banana', 'Lime', 'Pomegranate', 'Mango(Maldah)', 'Mango(Dushari)', 'Grapes(Green)', 'Grapes(Black)',
    'Water Melon(Green)', 'Sweet Orange', 'Pineapple', 'Jack Fruit', 'Cucumber(Local)', 'Cucumber(Hybrid)',
    'Papaya(Nepali)', 'Pear(Chinese)', 'Pear(Local)', 'Pear(Indian)', 'Litchi(Local)', 'Litchi(Indian)', 'Ginger',
    'Chilli Dry', 'Chilli Green', 'Chilli Green(Bullet)', 'Chilli Green(Machhe)', 'Chilli Green(Akbare)', 'Capsicum',
    'Garlic Green', 'Coriander Green', 'Garlic Dry Chinese', 'Garlic Dry Nepali', 'Clive Dry', 'Clive Green',
    'Fish Fresh(Rahu)', 'Fish Fresh(Bachuwa)', 'Fish Fresh(Chhadi)', 'Arum', 'Mushroom(Shiitake)'
]

In [14]:
len(english_list)

101

In [24]:
# Your full English list (136 items)
english_items = ['Amla', 'Apple(Fuji)', 'Apple(Jholey)', 'Arum', 'Asparagus',
       'Avocado', 'Bakula', 'Bamboo Shoot', 'Banana', 'Barela',
       'Bauhania flower', 'Bitter Gourd', 'Bottle Gourd',
       'Brd Leaf Mustard', 'Brinjal Long', 'Brinjal Round', 'Brocauli',
       'Cabbage', 'Cabbage(Local)', 'Cabbage(Terai)', 'Capsicum',
       'Carrot(Local)', 'Carrot(Terai)', 'Cauli Local',
       'Cauli Local(Jyapu)', 'Cauli Terai', 'Celery', 'Chilli Dry',
       'Chilli Green', 'Chilli Green(Akbare)', 'Chilli Green(Bullet)',
       'Chilli Green(Machhe)', 'Christophine', 'Clive Dry', 'Clive Green',
       'Coriander Green', 'Cow pea(Long)', 'Cowpea(Short)', 'Cress Leaf',
       'Cucumber(Hybrid)', 'Cucumber(Local)', 'Drumstick', 'Fennel Leaf',
       'Fenugreek Leaf', 'Fish Fresh', 'Fish Fresh(Bachuwa)',
       'Fish Fresh(Chhadi)', 'Fish Fresh(Mungari)', 'Fish Fresh(Rahu)',
       'French Bean(Hybrid)', 'French Bean(Local)', 'French Bean(Rajma)',
       'Garlic Dry Chinese', 'Garlic Dry Nepali', 'Garlic Green',
       'Ginger', 'Grapes(Black)', 'Grapes(Green)', 'Green Peas', 'Guava',
       'Gundruk', 'Jack Fruit', 'Kinnow', 'Kiwi', 'Knolkhol', 'Lemon',
       'Lettuce', 'Lime', 'Litchi(Indian)', 'Litchi(Local)', 'Maize',
       'Mandarin', 'Mango(Calcutte)', 'Mango(Chousa)', 'Mango(Dushari)',
       'Mango(Maldah)', 'Mint', 'Mombin', 'Mushroom(Button)',
       'Mushroom(Kanya)', 'Musk Melon', 'Mustard Leaf', 'Neuro', 'Okara',
       'Onion Dry (Chinese)', 'Onion Dry (Indian)', 'Onion Green',
       'Orange(Indian)', 'Orange(Nepali)', 'Papaya(Indian)',
       'Papaya(Nepali)', 'Parseley', 'Pear(Chinese)', 'Pear(Local)',
       'Pineapple', 'Pointed Gourd(Local)', 'Pointed Gourd(Terai)',
       'Pomegranate', 'Potato Red', 'Potato Red(Indian)',
       'Potato Red(Mude)', 'Potato White', 'Pumpkin', 'Raddish Red',
       'Raddish White(Hybrid)', 'Raddish White(Local)', 'Red Cabbbage',
       'Sarifa', 'Smooth Gourd', 'Snake Gourd', 'Soyabean Green',
       'Spinach Leaf', 'Sponge Gourd', 'Squash(Long)', 'Squash(Round)',
       'Strawberry', 'Sugarbeet', 'Sugarcane', 'Sweet Lime',
       'Sweet Orange', 'Sweet Potato', 'Sword Bean', 'Tamarind', 'Tofu',
       'Tomato Big(Indian)', 'Tomato Big(Nepali)', 'Tomato Small(Indian)',
       'Tomato Small(Local)', 'Tomato Small(Terai)',
       'Tomato Small(Tunnel)', 'Tree Tomato', 'Turnip', 'Turnip A',
       'Water Melon(Dotted)', 'Water Melon(Green)', 'Yam']

# Your dictionary (nepali_to_english_dict)

# Check for missing English items in the dictionary values:
missing_items = [item for item in english_items if item not in nepali_to_english_dict.values()]

if missing_items:
    print(f"Missing translations for these English items ({len(missing_items)}):")
    for miss in missing_items:
        print(miss)
else:
    print("All English items are properly mapped in the dictionary!")


All English items are properly mapped in the dictionary!


In [25]:
def translate_commodity(name):
    return nepali_to_english_dict.get(name, name)  # Replace if found, else keep original

df_kali['Commodity'] = df_kali['Commodity'].apply(translate_commodity)

In [34]:
def translate_commodity(name):
    name_stripped = name.strip()  # Remove leading/trailing spaces
    return nepali_to_english.get(name_stripped, name)  # Translate or keep original

df_kali['Commodity'] = df_kali['Commodity'].apply(translate_commodity)

# Find which items are still in Nepali (assuming they contain any non-ASCII chars)
still_nepali = [item for item in df_kali['Commodity'] if any(ord(ch) > 127 for ch in item)]
print("Still Nepali:", set(still_nepali))

Still Nepali: set()


In [33]:
nepali_to_english = {
    'बन्दा(नरिवल)': 'Squash(Long)',
    'चाक्सी': 'Okara',
    'टाटे सिमी': 'Cow pea(Long)',  # or 'Cowpea(Short)' depending on your use
    'नासपाती(लोकल)': 'Pear(Local)',
    'उखु': 'Sugarcane',
    'स्ट्रबेरी भुईऐसेलु': 'Strawberry',
    'आभोकाडो': 'Avocado',
    'तरबुजा(पाटे)': 'Water Melon(Green)',  # or Water Melon(Dotted)
    'तरुल': 'Pointed Gourd(Local)',  # or Pointed Gourd(Terai)
    'किवि': 'Kiwi',
    'कोइरालो': 'Bamboo Shoot',
    'लप्सी': 'Amla',
    'किनु': 'Kinnow',
    'काउली तराई': 'Cauli Local',
    'बकूला': 'Bakula',
    'आँप(चोसा)': 'Mango(Chousa)',
    'सुकेको प्याज (चाइनिज)': 'Garlic Dry Chinese',
    'शरीफा': 'Sarifa',
    'सलगम': 'Turnip',
    'रुख टमाटर': 'Tomato Big(Nepali)',  # or Tomato Big(Indian)
    'स्कूस': 'Sugarbeet',
    'स्थानीय काउली(ज्यापु)': 'Cauli Local'
}


In [35]:
df_kali

,Unnamed: 0,Date,Commodity,Unit,Minimum,Maximum,Average
0,0,2023-09-28,Tomato Big(Indian),kg,40.0,50.0,45.0
1,1,2023-09-28,Tomato Small(Local),kg,50.0,60.0,55.0
2,2,2023-09-28,Tomato Small(Tunnel),kg,60.0,75.0,69.0
3,3,2023-09-28,Tomato Small(Indian),kg,45.0,55.0,50.0
4,4,2023-09-28,Potato Red,kg,56.0,58.0,57.0
...,...,...,...,...,...,...,...
62112,62112,2025-06-07,Fish Fresh(Rahu),kg,310.0,330.0,320.0
62113,62113,2025-06-07,Fish Fresh(Bachuwa),kg,300.0,320.0,310.0
62114,62114,2025-06-07,Fish Fresh(Chhadi),kg,220.0,240.0,230.0
62115,62115,2025-06-07,Arum,kg,280.0,300.0,290.0


In [37]:
df_kali.Commodity.unique()

array(['Tomato Big(Indian)', 'Tomato Small(Local)',
       'Tomato Small(Tunnel)', 'Tomato Small(Indian)', 'Potato Red',
       'Potato Red(Indian)', 'Potato Red(Mude)', 'Potato White',
       'Onion Dry (Indian)', 'Carrot(Local)', 'Carrot(Terai)',
       'Cabbage(Local)', 'Cauli Local', 'Raddish Red',
       'Raddish White(Local)', 'Raddish White(Hybrid)', 'Brinjal Long',
       'Brinjal Round', 'Cow pea(Long)', 'Maize', 'French Bean(Local)',
       'French Bean(Hybrid)', 'Soyabean Green', 'Bitter Gourd',
       'Bottle Gourd', 'Pointed Gourd(Local)', 'Pointed Gourd(Terai)',
       'Snake Gourd', 'Smooth Gourd', 'Pumpkin', 'Squash(Round)', 'Okara',
       'Barela', 'Brd Leaf Mustard', 'Sugarbeet', 'Spinach Leaf',
       'Mustard Leaf', 'Onion Green', 'Mushroom(Kanya)',
       'Mushroom(Button)', 'Asparagus', 'Neuro', 'Brocauli',
       'Red Cabbbage', 'Lettuce', 'Celery', 'Parseley', 'Mint',
       'Turnip A', 'Tamarind', 'Bamboo Shoot', 'Tofu', 'Gundruk',
       'Apple(Jholey)', 'App

In [42]:
df_kali.to_csv('kalimati_prices.csv',index=False)

In [40]:
df_kali.drop('Unnamed: 0',inplace=True,axis=1)